1. Videos Scenes visual embeddings
2. Video Scenes text (caption) embeddings
-----------------------------------------------------
3. Video Visual embeddings (all frames) + Averaging
4. Video text (captions) embeddings
-----------------------------------------------------
5. Video visual embeddings (Down-sampled frames)
6. Video text embeddings (Down-sampled frames)
-----------------------------------------------------

A. Mode

B. Medoid

C. Random

* Feature-based fusion
* Score-based fusion


In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import Counter
from collections import defaultdict
import random
import statistics
import pickle as pkl
import os
import random
from scipy.spatial.distance import cdist
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/Finalllll/MSRVTT_all_embs&augmentation.pkl", "rb") as f:
    msrvtt_dataset_all_embs = pkl.load(f)

In [ ]:
with open("/content/drive/MyDrive/Finalllll/MSVD_all_embs&augmentation.pkl", "rb") as f:
    MSVD_all_embs = pkl.load(f)

In [ ]:
def find_mode(numbers):
    if not numbers:
        return None  # Return None if the list is empty

    count = Counter(numbers)  # Count occurrences of each element
    max_frequency = max(count.values())  # Find the highest frequency

    # Get all elements with the maximum frequency
    modes = [key for key, val in count.items() if val == max_frequency]

    return min(modes)  # Return the smallest mode

def agg_embs(embeddings):
    for video_id, embs in embeddings.items():
        try:
            n = len(embs)
        except:
            print("This dictionary does not have multiple embeddings")
            return embeddings, False
        embs = torch.stack(embs)
        embs = torch.sum(embs, dim=0)
        embs = embs / n
        embeddings[video_id] = embs
    return embeddings, True

def compute_similarities_parallel(emb_dict, emb, device=device):
    sim_current_emb_avg = []
    sim_current_emb_max = []

    # Move the reference embedding to the device (GPU or CPU)
    emb = emb.to(device)

    # Iterate over each video in emb_dict
    for v_id, embeddings in emb_dict.items():
        n = len(embeddings)
        # Stack embeddings for the current video into a tensor
        embeddings_tensor = torch.stack([embedding.to(device) for embedding in embeddings])

        # Compute the dot product (similarities) in a batch-wise manner
        sim_matrix = torch.mm(embeddings_tensor.squeeze(dim=1), emb.T)  # Compute all pairwise similarities at once

        # Sum similarities across embeddings
        total_sims = sim_matrix.sum().item()

        # Get the maximum similarity for the current video
        max_sim_current_vid = sim_matrix.max().item()

        # Store results
        sim_current_emb_avg.append([v_id, total_sims / n])
        sim_current_emb_max.append([v_id, max_sim_current_vid])

    return sim_current_emb_avg, sim_current_emb_max

def compute_rank(vids_sims, video_id):
    # Sort by similarity values in descending order
    sorted_vids_sims = sorted(vids_sims, key=lambda x: x[1], reverse=True)
    # Create dictionaries to map video_id to its rank
    rank_vids = {video[0]: i + 1 for i, video in enumerate(sorted_vids_sims)}
    # Get the ranks for the current video using dictionaries
    rank = rank_vids.get(video_id, -1)
    return rank

def get_medoid_index(embs_list):
    # Convert the list of embeddings to a single tensor of shape [N, 512]
    embeddings_tensor = torch.vstack(embs_list)
    # Compute pairwise distances (Euclidean distance)
    pairwise_distances = torch.cdist(embeddings_tensor, embeddings_tensor, p=2)
    # Sum distances for each embedding
    sum_distances = torch.sum(pairwise_distances, dim=1)
    # Find the index of the medoid
    medoid_index = torch.argmin(sum_distances)
    return medoid_index

def calc_single_emb_measures(result):
    measures = {}
    for test in result:
        key = list(test.keys())[0]
        measures[key] = {}
        mode_ranks = []
        random_ranks = []
        medoid_ranks = []
        median_ranks = []
        for id in test[key]:

            mode_ranks.append(test[key][id]["mode_rank"])
            random_ranks.append(test[key][id]["random_rank"])
            medoid_ranks.append(test[key][id]["medoid_rank"])
            median_ranks.append(test[key][id]["median_rank"])
        n = len(test[key])
        ks = [1, 5, 10]
        for k in ks:
            measures[key][f"R@{k}_mode"] = 0
            for rank in mode_ranks:
                if rank <= k:
                    measures[key][f"R@{k}_mode"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_median"] = 0
            for rank in median_ranks:
                if rank <= k:
                    measures[key][f"R@{k}_median"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_medoid"] = 0
            for rank in medoid_ranks:
                if rank <= k:
                    measures[key][f"R@{k}_medoid"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_random"] = 0
            for rank in random_ranks:
                if rank <= k:
                    measures[key][f"R@{k}_random"] += 1 / n
        measures[key]["MdR_mode"] = statistics.median(mode_ranks)
        measures[key]["MnR_mode"] = sum(mode_ranks) / n
        measures[key]["MdR_median"] = statistics.median(median_ranks)
        measures[key]["MnR_median"] = sum(median_ranks) / n
        measures[key]["MdR_medoid"] = statistics.median(medoid_ranks)
        measures[key]["MnR_medoid"] = sum(medoid_ranks) / n
        measures[key]["MdR_random"] = statistics.median(random_ranks)
        measures[key]["MnR_random"] = sum(random_ranks) / n
    return measures


def calc_multiple_embs_measures(result):
    measures = {}
    for test in result:
        key = list(test.keys())[0]
        measures[key] = {}
        mode_ranks_avg = []
        random_ranks_avg = []
        medoid_ranks_avg = []
        mode_ranks_max = []
        random_ranks_max = []
        medoid_ranks_max = []
        for id in test[key]:
            mode_ranks_avg.append(test[key][id]["mode_rank_avg"])
            random_ranks_avg.append(test[key][id]["random_rank_avg"])
            medoid_ranks_avg.append(test[key][id]["medoid_rank_avg"])
            mode_ranks_max.append(test[key][id]["mode_rank_max"])
            random_ranks_max.append(test[key][id]["random_rank_max"])
            medoid_ranks_max.append(test[key][id]["medoid_rank_max"])
        n = len(test[key])
        ks = [1, 5, 10]
        for k in ks:
            measures[key][f"R@{k}_mode_avg"] = 0
            for rank in mode_ranks_avg:
                if rank <= k:
                    measures[key][f"R@{k}_mode_avg"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_medoid_avg"] = 0
            for rank in medoid_ranks_avg:
                if rank <= k:
                    measures[key][f"R@{k}_medoid_avg"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_random_avg"] = 0
            for rank in random_ranks_avg:
                if rank <= k:
                    measures[key][f"R@{k}_random_avg"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_mode_max"] = 0
            for rank in mode_ranks_max:
                if rank <= k:
                    measures[key][f"R@{k}_mode_max"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_medoid_max"] = 0
            for rank in medoid_ranks_max:
                if rank <= k:
                    measures[key][f"R@{k}_medoid_max"] += 1 / n
        for k in ks:
            measures[key][f"R@{k}_random_max"] = 0
            for rank in random_ranks_max:
                if rank <= k:
                    measures[key][f"R@{k}_random_max"] += 1 / n
        measures[key]["MdR_mode_avg"] = statistics.median(mode_ranks_avg)
        measures[key]["MnR_mode_avg"] = sum(mode_ranks_avg) / n
        measures[key]["MdR_medoid_avg"] = statistics.median(medoid_ranks_avg)
        measures[key]["MnR_medoid_avg"] = sum(medoid_ranks_avg) / n
        measures[key]["MdR_random_avg"] = statistics.median(random_ranks_avg)
        measures[key]["MnR_random_avg"] = sum(random_ranks_avg) / n
        measures[key]["MdR_mode_max"] = statistics.median(mode_ranks_max)
        measures[key]["MnR_mode_max"] = sum(mode_ranks_max) / n
        measures[key]["MdR_medoid_max"] = statistics.median(medoid_ranks_max)
        measures[key]["MnR_medoid_max"] = sum(medoid_ranks_max) / n
        measures[key]["MdR_random_max"] = statistics.median(random_ranks_max)
        measures[key]["MnR_random"] = sum(random_ranks_max) / n
    return measures


def combine_sims(sims_1, sims_2, alpha):
    final_dict = {}
    for item in sims_1:

        final_dict[item[0]] = item[1] * alpha
    for item in sims_2:

        final_dict[item[0]] += item[1] * (1 - alpha)

    return [[key, value] for key, value in final_dict.items()]

In [ ]:
def combine_both_embs(text_embeddings, visual_embeddings, mode, alpha):
    final_embs = {}
    multiple_embs = False
    if mode == "both_multiple":
        for id in text_embeddings:
            final_embs[id] = []
            for i in range(len(text_embeddings[id])):
                final_embs[id].append(alpha * text_embeddings[id][i].to(device) + (1 - alpha) * visual_embeddings[id][i].to(device))
        multiple_embs = True
    else:
        for id in text_embeddings:
            final_embs[id] = alpha * text_embeddings[id].to(device) + (1 - alpha) * visual_embeddings[id].to(device)
        multiple_embs = False
    return final_embs, multiple_embs

def get_fusion_embs(text_embeddings, visual_embeddings, agg_text_embs=False, agg_visual_embs=False, alpha=0.5):
    text_has_multiple_embs = False
    visual_has_multiple_embs = False
    if agg_text_embs:
        text_embeddings, text_has_multiple_embs = agg_embs(text_embeddings.copy())
    if agg_visual_embs:
        visual_embeddings, visual_has_multiple_embs = agg_embs(visual_embeddings.copy())
    if type(list(text_embeddings.values())[0]) == list:
        text_has_multiple_embs = True
    else:
        text_has_multiple_embs = False
    if type(list(visual_embeddings.values())[0]) == list:
        visual_has_multiple_embs = True
    else:
        visual_has_multiple_embs = False
    if text_has_multiple_embs and visual_has_multiple_embs:
        mode = "both_multiple"
    elif text_has_multiple_embs:
        mode = "text_only"
    elif visual_has_multiple_embs:
        mode = "visual_only"
    else:
        mode = "Neither"
    return combine_both_embs(text_embeddings, visual_embeddings, mode, alpha)

def calc_sim_one_dict(emb_dict, queries_embs):
    has_multiple = False
    if type(list(emb_dict.values())[0]) == list:
        has_multiple = True
    counter = 1
    ranks_per_video = {}
    if has_multiple:

        ranks_mode_max = {}
        ranks_random_max = {}
        ranks_medoid_max = {}
        ranks_mode_avg = {}
        ranks_random_avg = {}
        ranks_medoid_avg = {}
        for video_id in queries_embs:
            ranks_per_video[video_id] = {}
            print(counter)
            counter += 1
            ranks_current_video_avg = []
            ranks_current_video_max = []
            embs_list = queries_embs[video_id]
            medoid_index = get_medoid_index(embs_list)
            random_index = random.randint(0, len(embs_list) - 1)
            for emb in embs_list:
                sim_current_emb_avg, sim_current_emb_max = compute_similarities_parallel(emb_dict, emb, device)
                rank_avg = compute_rank(sim_current_emb_avg, video_id)
                rank_max = compute_rank(sim_current_emb_max, video_id)
                ranks_current_video_max.append(rank_max)
                ranks_current_video_avg.append(rank_avg)

            ranks_per_video[video_id]["mode_rank_avg"] = find_mode(ranks_current_video_avg)
            ranks_per_video[video_id]["mode_rank_max"] = find_mode(ranks_current_video_max)

            ranks_per_video[video_id]["medoid_rank_avg"] = ranks_current_video_avg[medoid_index]
            ranks_per_video[video_id]["medoid_rank_max"] = ranks_current_video_max[medoid_index]
            ranks_per_video[video_id]["random_rank_avg"] = ranks_current_video_avg[random_index]
            ranks_per_video[video_id]["random_rank_max"] = ranks_current_video_max[random_index]
            ranks_per_video[video_id]["all_ranks_max"] = ranks_current_video_max
            ranks_per_video[video_id]["all_ranks_avg"] = ranks_current_video_avg
        return ranks_per_video
    else:
        ranks_mode   = {}
        ranks_random = {}
        ranks_medoid = {}
        for video_id in queries_embs:
            print(counter)
            counter += 1
            ranks_per_video[video_id] = {}
            ranks_current_video = []
            embs_list = queries_embs[video_id]
            medoid_index = get_medoid_index(embs_list)
            random_index = random.randint(0, len(embs_list) - 1)
            for emb in embs_list:
                sim_current_emb = []
                all_embs = torch.cat([emb_dict[v_id] for v_id in emb_dict], dim=0).to(device)  # Shape: (N, 512)
                # Compute similarity scores in one matrix multiplication
                # emb is of shape (1, 512), so no need to unsqueeze
                similarities = torch.mm(emb.to(device), all_embs.T).squeeze(0)  # Shape: (N,)
                # Prepare the result as a list of [v_id, similarity]
                sim_current_emb = [[v_id, sim.item()] for v_id, sim in zip(emb_dict.keys(), similarities)]
                rank = compute_rank(sim_current_emb, video_id)
                ranks_current_video.append(rank)
            ranks_per_video[video_id]["mode_rank"] = find_mode(ranks_current_video)
            ranks_per_video[video_id]["medoid_rank"] = ranks_current_video[medoid_index]
            ranks_per_video[video_id]["random_rank"] = ranks_current_video[random_index]
            ranks_per_video[video_id]["all_ranks"] = ranks_current_video
        return ranks_per_video

def score_based_fusion_test(emb_dict_1, emb_dict_2, queries_embs, alpha=0.5):
    has_multiple = False
    if type(list(emb_dict_1.values())[0]) == list:
        has_multiple = True
    counter = 1
    ranks_per_video = {}
    if has_multiple:
        ranks_mode_max = {}
        ranks_random_max = {}
        ranks_medoid_max = {}
        ranks_mode_avg = {}
        ranks_random_avg = {}
        ranks_medoid_avg = {}
        for video_id in queries_embs:
            print(counter)
            counter += 1
            ranks_per_video[video_id] = {}
            ranks_current_video_avg = []
            ranks_current_video_max = []
            embs_list = queries_embs[video_id]
            medoid_index = get_medoid_index(embs_list)
            random_index = random.randint(0, len(embs_list) - 1)
            for emb in embs_list:
                sim_current_emb_avg_1, sim_current_emb_max_1 = compute_similarities_parallel(emb_dict_1, emb, device)
                sim_current_emb_avg_2, sim_current_emb_max_2 = compute_similarities_parallel(emb_dict_2, emb, device)
                sim_avg = combine_sims(sim_current_emb_avg_1, sim_current_emb_avg_2, alpha)
                sim_max = combine_sims(sim_current_emb_max_1, sim_current_emb_max_2, alpha)
                rank_avg = compute_rank(sim_avg, video_id)
                rank_max = compute_rank(sim_max, video_id)
                ranks_current_video_max.append(rank_max)
                ranks_current_video_avg.append(rank_avg)
            ranks_per_video[video_id]["mode_rank_avg"] = find_mode(ranks_current_video_avg)
            mode_rank = find_mode(ranks_current_video_max)

            ranks_per_video[video_id]["mode_rank_max"] = mode_rank
            ranks_per_video[video_id]["medoid_rank_avg"] = ranks_current_video_avg[medoid_index]
            ranks_per_video[video_id]["medoid_rank_max"] = ranks_current_video_max[medoid_index]
            ranks_per_video[video_id]["random_rank_avg"] = ranks_current_video_avg[random_index]
            ranks_per_video[video_id]["random_rank_max"] = ranks_current_video_max[random_index]
            ranks_per_video[video_id]["all_ranks_max"] = ranks_current_video_max
            ranks_per_video[video_id]["all_ranks_avg"] = ranks_current_video_avg
        return ranks_per_video
    else:
        ranks_mode = {}
        ranks_random = {}
        ranks_medoid = {}
        for video_id in queries_embs:
            print(counter)
            counter += 1
            ranks_per_video[video_id] = {}
            ranks_current_video = []
            embs_list = queries_embs[video_id]
            medoid_index = get_medoid_index(embs_list)
            random_index = random.randint(0, len(embs_list) - 1)
            for emb in embs_list:
                sim_current_emb = []
                for v_id in emb_dict_1.keys():
                    emb_1 = emb_dict_1[v_id].to(device)
                    emb_2 = emb_dict_2.get(v_id, torch.zeros_like(emb_1)).to(device)  # Default to zero if v_id is missing

                    # Compute similarity
                    sim_1 = torch.cosine_similarity(emb, emb_1).item()  # Cosine similarity
                    sim_2 = torch.cosine_similarity(emb, emb_2).item()

                    # Weighted similarity
                    weighted_sim = alpha * sim_1 + (1 - alpha) * sim_2

                    sim_current_emb.append([v_id, weighted_sim])
                rank = compute_rank(sim_current_emb, video_id)
                ranks_current_video.append(rank)
            mode_rank = find_mode(ranks_current_video)

            median_rank = round(np.median(ranks_current_video))
            print(f"Rank: {median_rank}")
            ranks_per_video[video_id]["median_rank"] = median_rank
            ranks_per_video[video_id]["mode_rank"] = mode_rank
            ranks_per_video[video_id]["medoid_rank"] = ranks_current_video[medoid_index]
            ranks_per_video[video_id]["random_rank"] = ranks_current_video[random_index]
            ranks_per_video[video_id]["all_ranks"] = ranks_current_video

        return ranks_per_video
def do_experiments(mode, multiple, text_embs, visual_embs, queries_embs, feature_based_fusion=False):
    experiment = ""
    experiment_result = {}
    if mode == 1:
        experiment = "Text embs only"
        print(experiment)
        experiment_result[experiment] = calc_sim_one_dict(text_embs, queries_embs)
        if multiple:
            text_embs_agg, _ = agg_embs(text_embs)
            experiment += " aggregated"
            experiment_result[experiment] = calc_sim_one_dict(text_embs_agg, queries_embs)

    elif mode == 2:
        experiment = "Visual embs only"
        print(experiment)
        experiment_result[experiment] = calc_sim_one_dict(visual_embs, queries_embs)

        if multiple:
            experiment += " aggregated"
            visual_embs_agg, _ = agg_embs(visual_embs)
            print(experiment)
            experiment_result[experiment] = calc_sim_one_dict(visual_embs_agg, queries_embs)

    elif mode == 3:
        if multiple:
            for choice in [True, False]:
                    experiment = "Feature-based fusion"
                    if choice:
                        experiment += " aggregate both"
                    else:
                        experiment += " no aggregations"
                    print(experiment)
                    final_combination, _ = get_fusion_embs(text_embs, visual_embs, agg_text_embs=choice, agg_visual_embs=choice, alpha=0.5)
                    experiment_result[experiment] = calc_sim_one_dict(final_combination, queries_embs)
        else:
                experiment += " single embeding for each one"
                print(experiment)
                final_combination, _ = get_fusion_embs(text_embs, visual_embs, agg_text_embs=False, agg_visual_embs=False, alpha=0.5)
                experiment_result[experiment] = calc_sim_one_dict(final_combination, queries_embs)
    return experiment_result

def do_experiments_frames(dataset, augment=False):
    multiple = True

    choice = "Frames"
    if augment:
        print("Augmentation!")
        queries_embs = dataset["augmented_captions"]
    else:
        queries_embs = dataset["Ground truth captions"]
    result = []
    text_embs = dataset[choice]["Text"]
    visual_embs = dataset[choice]["CLIP"]
    for i in range(1, 4):
        mode = i
        result.append(do_experiments(mode, multiple, text_embs.copy(), visual_embs.copy(), queries_embs))
    return result
def do_experiments_videos(dataset, augment=False):
    multiple = False


    choice = "Videos"
    if augment:
        print("Augmentation!")
        queries_embs = dataset["augmented_captions"]
    else:
        queries_embs = dataset["Ground truth captions"]
    result = []
    text_embs = dataset[choice]["Text"].copy()
    visual_embs = dataset[choice]["CLIP"].copy()
    for i in range(1, 4):
        mode = i
        result.append(do_experiments(mode, multiple, text_embs.copy(), visual_embs.copy(), queries_embs))
    return result
def do_experiments_scenes(dataset, augment=False):
    multiple = True

    choice = "Scenes"
    if augment:
        print("Augmentation!")
        queries_embs = dataset["augmented_captions"]
    else:
        queries_embs = dataset["Ground truth captions"]
    result = []
    text_embs = dataset[choice]["Text"]
    visual_embs = dataset[choice]["CLIP"]
    for i in range(1, 4):
        mode = i
        result.append(do_experiments(mode, multiple, text_embs.copy(), visual_embs.copy(), queries_embs))
    return result


In [ ]:
text_embs, _ = agg_embs(msrvtt_dataset_all_embs["Frames"]["Text"].copy())
visual_embs, _ = agg_embs(msrvtt_dataset_all_embs["Frames"]["CLIP"].copy())
queries_embs = msrvtt_dataset_all_embs["Ground truth captions"]

In [ ]:
len(text_embs)

1000

In [ ]:
results = score_based_fusion_test(text_embs, visual_embs, queries_embs, alpha=0.5)

1
Rank: 8
2
Rank: 603
3
Rank: 3
4
Rank: 100
5
Rank: 1
6
Rank: 8
7
Rank: 2
8
Rank: 456
9
Rank: 3
10
Rank: 23
11
Rank: 5
12
Rank: 2
13
Rank: 57
14
Rank: 4
15
Rank: 12
16
Rank: 2
17
Rank: 1
18
Rank: 2
19
Rank: 210
20
Rank: 3
21
Rank: 1
22
Rank: 364
23
Rank: 1
24
Rank: 14
25
Rank: 30
26
Rank: 38
27
Rank: 177
28
Rank: 28
29
Rank: 2
30
Rank: 2
31
Rank: 702
32
Rank: 2
33
Rank: 2
34
Rank: 1
35
Rank: 80
36
Rank: 3
37
Rank: 3
38
Rank: 1
39
Rank: 1
40
Rank: 22
41
Rank: 24
42
Rank: 12
43
Rank: 36
44
Rank: 1
45
Rank: 240
46
Rank: 124
47
Rank: 14
48
Rank: 10
49
Rank: 1
50
Rank: 92
51
Rank: 27
52
Rank: 32
53
Rank: 1
54
Rank: 1
55
Rank: 1
56
Rank: 17
57
Rank: 26
58
Rank: 14
59
Rank: 440
60
Rank: 7
61
Rank: 68
62
Rank: 46
63
Rank: 924
64
Rank: 68
65
Rank: 1
66
Rank: 100
67
Rank: 1
68
Rank: 631
69
Rank: 2
70
Rank: 44
71
Rank: 136
72
Rank: 2
73
Rank: 1
74
Rank: 34
75
Rank: 38
76
Rank: 5
77
Rank: 1
78
Rank: 24
79
Rank: 432
80
Rank: 8
81
Rank: 298
82
Rank: 42
83
Rank: 18
84
Rank: 17
85
Rank: 18
86
Rank: 3


In [ ]:
import json
with open("results_frames_no_augmentation_sbf.json", "w") as f:
    json.dump(results, f, indent=4)

In [ ]:
results_test = {}
results_test["Score based fusion"] = results
test = [results_test]

In [ ]:
measures = calc_single_emb_measures(test)


In [ ]:
measures

{'Score based fusion': {'R@1_mode': 0.5150000000000003,
  'R@5_mode': 0.7120000000000005,
  'R@10_mode': 0.7670000000000006,
  'R@1_median': 0.17300000000000013,
  'R@5_median': 0.3870000000000003,
  'R@10_median': 0.4910000000000004,
  'R@1_medoid': 0.23500000000000018,
  'R@5_medoid': 0.4160000000000003,
  'R@10_medoid': 0.4840000000000004,
  'R@1_random': 0.24000000000000019,
  'R@5_random': 0.4170000000000003,
  'R@10_random': 0.4900000000000004,
  'MdR_mode': 1.0,
  'MnR_mode': 51.421,
  'MdR_median': 12.0,
  'MnR_median': 85.662,
  'MdR_medoid': 12.0,
  'MnR_medoid': 109.381,
  'MdR_random': 11.0,
  'MnR_random': 110.513}}

In [ ]:
with open("measures_frames_no_augmentation_sbf.json", "w") as f:
    json.dump(measures, f, indent=4)